In [7]:
import sys
sys.path.append("agsr_net") 
from agsr_net import preprocessing
from agsr_net.model import AGSRNet
from agsr_net.train import train, test
import argparse
from sklearn.model_selection import KFold
from MatrixVectorizer import MatrixVectorizer
import pandas as pd
import numpy as np



In [3]:
class Args:
    def __init__(self):
        self.epochs = 200  
        self.lr = 0.0001   
        self.lmbda = 0.1 
        self.lr_dim = 160     
        self.hr_dim = 268  
        self.hidden_dim = 320 
        self.padding = 26 
        self.mean_dense = 0.0    
        self.std_dense = 0.01      
        self.mean_gaussian = 0.0  
        self.std_gaussian = 0.1

ks = [0.9, 0.7, 0.6, 0.5]
args = Args()

In [4]:
LR_TRAIN_DATA_PATH = "data/lr_train.csv"
LR_TEST_DATA_PATH = "data/lr_test.csv"
HR_TRAIN_DATA_PATH = "data/hr_train.csv"

df_lr_train = pd.read_csv(LR_TRAIN_DATA_PATH)
df_lr_test = pd.read_csv(LR_TEST_DATA_PATH)
df_hr_train = pd.read_csv(HR_TRAIN_DATA_PATH)

v_lr_train = np.zeros((len(df_lr_train), 160, 160))
v_lr_test = np.zeros((len(df_lr_test), 160, 160))
v_hr_train = np.zeros((len(df_hr_train), 268, 268))

mv = MatrixVectorizer()

for i, row in enumerate(df_lr_train.values):
    v_lr_train[i] = mv.anti_vectorize(row, 160)

for i, row in enumerate(df_lr_test.values):
    v_lr_test[i] = mv.anti_vectorize(row, 160)

for i, row in enumerate(df_hr_train.values):
    v_hr_train[i] = mv.anti_vectorize(row, 268)


In [9]:
k_folds = 3
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

X = v_lr_train
y = v_hr_train

fold_metrics = []
for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"Fold {fold + 1}/{k_folds}")

    # Split data for this fold
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    print(X_train.shape)    
    print(y_train.shape)

    print(type(X_train))


    print(f"  Training samples: {X_train.shape[0]}")
    print(f"  Validation samples: {X_val.shape[0]}")

    # print(X_train.shape)
    # print(y_train.shape)
    # model = model(
    #         input_dim=12720,
    #         hidden_dims=[4096, 2048, 1024, 512],
    #         output_dim=35778,
    #         dropout_rate=0.3
    # )

    model = AGSRNet(ks, args)




    train(model, X_train, y_train, args)
    test(model, X_val, y_val, args)
    break

    # model, metrics = train.train(model, X_train, y_train, X_val, y_val)
    # fold_metrics.append(metrics)

    # print(f"  Validation Loss: {metrics['val_loss']:.4f}")
# print(fold_metrics)





Fold 1/3
(111, 160, 160)
(111, 268, 268)
<class 'numpy.ndarray'>
  Training samples: 111
  Validation samples: 56
Discriminator(
  (dense_1): Dense()
  (relu_1): ReLU()
  (dense_2): Dense()
  (relu_2): ReLU()
  (dense_3): Dense()
  (sigmoid): Sigmoid()
)


RuntimeError: This function was deprecated since version 1.9 and is now removed. The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.

L, _ = torch.symeig(A, upper=upper) should be replaced with:
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')

and

L, V = torch.symeig(A, eigenvectors=True) should be replaced with:
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L')